# Czech 2050 nuclear scenarios analysis


## Imports


In [ ]:
# Add parent dir to the path, so we can load the module

import os
import sys
from typing import Optional

sys.path.append(os.path.dirname(os.path.abspath("")))

from energy_insights.execution_utils import optimize_runs, get_extrapolator
from energy_insights.loaders.ember_ng import EmberNgLoader
from energy_insights.region import *
from energy_insights.scenarios.czech_nuclear import (
    construct_grid, make_nuclear_scenarios, make_original_scenarios
)


## Config


In [ ]:
load_hydro_from_pecd = True
load_demand_from_pecd = True
common_year = 2008
pecd_year = 2008
entsoe_year = 2020  # ENTSO-E has only data back to 2010, we need a fallback year for the load.
aggregation_level: Optional[str] = "coarse"

extrapolator = get_extrapolator(data_path="../data/")

ember_loader = EmberNgLoader(
    entsoe_year=entsoe_year,
    pecd_year=pecd_year,
    common_year=common_year,
    data_file="../data/ember/Raw data - New Generation.xlsx",
    tyndp_input_file="../data/tyndp/Draft_Demand_Scenarios_TYNDP_2024.xlsb",
    load_hydro_from_pecd=load_hydro_from_pecd,
    load_demand_from_pecd=load_demand_from_pecd,
    extrapolator=extrapolator)


def make_nuclear_run(ember_scenario: str,
                     target_year_europe: int,
                     target_year_cz: int) -> dict:
    short_name: str = ember_scenario.lower().replace(" ", "-")

    return {
        "config": {
            "common_year": common_year,
            "entsoe_year": entsoe_year,
            "pecd_year": pecd_year,
            "load_hydro_from_pecd": load_hydro_from_pecd,
            "load_demand_from_pecd": load_demand_from_pecd,
            "analysis_name": f"ember-{short_name}",
            "filter": {
                "week_sampling": 4,  # Plot every fourth week in the output.
                "countries": [CZECHIA],
                # "days": [
                #     "2020-07-08", "2020-07-09", "2020-07-10",
                #     "2020-11-25", "2020-11-26", "2020-11-27"
                # ],
            },
            "output": {
                # "max_gw": 17,
                # "min_gw": -5.5,
                # "size_x_week": 0.8,
                "format": "png",
                # "format": "svg",
                "dpi": 150,
                "size_y_week": 0.7,
                "parts": ["titles", "weeks", "week_summary", "year_stats"],
                # "parts": ["weeks"],
                "regions": "separate",
            },
            "optimize_capex": True,
            "input_costs": "2050-SEK",
            "optimize_ramp_up_costs": True,
            # "load_previous_solution": True,
            "import_ppa_price": 60,
            # "solver": "Mosek",
            "store_model": False,
        } | construct_grid(ember_loader, ember_scenario, target_year_europe,
                           aggregation_level=aggregation_level),
        "scenarios": make_nuclear_scenarios(ember_loader, ember_scenario, target_year_cz),
        # "scenarios": make_original_scenario(ember_loader, ember_scenario, target_year_europe),
    }


In [ ]:
runs = [
    # Nuclear analysis scenarios:
    # make_nuclear_run(EmberNgLoader.STATED_POLICY, 2050, 2050),
    # make_nuclear_run(EmberNgLoader.TECHNOLOGY_DRIVEN_BATTERY, 2050, 2050),

    # Optimistic sensitivity scenarios (in terms of build-out / progress).
    # make_nuclear_run(EmberNgLoader.NUCLEAR_PLUS, 2050, 2050),

    # Pessimistic sensitivity scenarios, 2040 to strengthen the effect.
    # make_nuclear_run(EmberNgLoader.DELAYED_INTERCONNECTIONS, 2040, 2050),
    make_nuclear_run(EmberNgLoader.NO_GAS_WITH_CCS, 2040, 2050),
    # make_nuclear_run(EmberNgLoader.RESISTANCE_TO_RES, 2040, 2050),

    # Excluded scenarios:
    # - core pathways:
    # make_nuclear_run(EmberNgLoader.SYSTEM_CHANGE, 2050, 2050),
    # - optimistic sensitivity scenarios:
    # make_nuclear_run(EmberNgLoader.SYSTEM_CHANGE_BATTERY, 2050, 2050),
    # make_nuclear_run(EmberNgLoader.TECHNOLOGY_DRIVEN_BATTERY, 2050, 2050),
    # make_nuclear_run(EmberNgLoader.ALTERNATIVE_HYDROGEN_SUPPLY, 2050, 2050),
    # - pessimistic sensitivity scenarios:
    # make_nuclear_run(EmberNgLoader.HIGHER_FOSSIL_FUEL_PRICES, 2040, 2050),
    # make_nuclear_run(EmberNgLoader.LIMITED_NEW_GAS, 2040, 2050),
    # make_nuclear_run(EmberNgLoader.LOWER_DEMAND_FLEXIBILITY, 2040, 2050),
]


## Execution


In [ ]:
optimize_runs(runs, extrapolator=extrapolator)